In [34]:
from moco.builder import MoCo
import torch
from torchvision import models

moco=MoCo(
    models.__dict__['resnet18'],
    128, 65536, 0.999,
    contr_tau=0.2,
    align_alpha=None,
    unif_t=None,
    unif_intra_batch=True,
    mlp=True)
moco=moco.cuda()
checkpoint = torch.load('../moco/save/custom_imagenet_n02106550/mocom0.999_contr1tau0.2_mlp_aug+_cos_b256_lr0.06_e120,160,200/checkpoint_0199.pth.tar', map_location=torch.device('cuda:0'))

# moco=moco.load_state_dict(checkpoint['state_dict'])
# moco=checkpoint

In [35]:
# moco=moco.load_state_dict(checkpoint['state_dict'],strict=False)

In [36]:
# moco

In [37]:
state_dict =checkpoint['state_dict']
from collections import OrderedDict
new_state_dict = OrderedDict()

for k, v in state_dict.items():
    if 'module' in k:
        k = k.split('.')[1:]
        k = '.'.join(k)
    new_state_dict[k]=v

moco.load_state_dict(new_state_dict)

<All keys matched successfully>

In [39]:
moco.encoder_q

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights,vit_l_16,ViT_L_16_Weights,vit_b_16,ViT_B_16_Weights,swin_s,Swin_S_Weights

# self.backbone=vit_l_16(weights=ViT_L_16_Weights.IMAGENET1K_SWAG_LINEAR_V1).to(self.device).eval()
# backbone=vit_b_16(weights=ViT_B_16_Weights.IMAGENET1K_SWAG_LINEAR_V1)

# backbone=swin_s(weights=Swin_S_Weights.IMAGENET1K_V1)


In [13]:
from torch.nn import Identity
backbone.head=Identity()

In [5]:
backbone

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
from torch.nn import Identity
backbone.head=Identity()

In [ ]:
import sys,torch
sys.path.append('..')

from simclr_converter.resnet_wider import resnet50x1, resnet50x2, resnet50x4

model = resnet50x2()
sd = '../simclr_converter/resnet50-2x.pth'

sd = torch.load(sd, map_location='cpu')
model.load_state_dict(sd['state_dict'])

model

In [66]:
import torch
import torchvision.models as models
import torch.nn as nn

model = models.__dict__['resnet50']()

checkpoint = torch.load('/home/hrzhang/projects/SSL-Backdoor/moco/save/moco_v2_800ep_pretrain.pth.tar')
state_dict = checkpoint["state_dict"]

for k in list(state_dict.keys()):
    # retain only encoder_q up to before the embedding layer
    if k.startswith("module.encoder") and not k.startswith(
        "module.encoder_q.fc"
    ):
        # remove prefix
        state_dict[k[len("module.encoder_q.") :]] = state_dict[k]
    # delete renamed or unused k
    del state_dict[k]

model.load_state_dict(state_dict, strict=False)

_IncompatibleKeys(missing_keys=['fc.weight', 'fc.bias'], unexpected_keys=[])

In [67]:
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 1024),
    nn.ReLU(),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Linear(256, 128),
)

for name, param in model.named_parameters():
    if not name.startswith("fc"):
        param.requires_grad = False


In [68]:
for name, param in model.named_parameters():
    print(f'Parameter: {name}, Requires Grad: {param.requires_grad}')

Parameter: conv1.weight, Requires Grad: False
Parameter: bn1.weight, Requires Grad: False
Parameter: bn1.bias, Requires Grad: False
Parameter: layer1.0.conv1.weight, Requires Grad: False
Parameter: layer1.0.bn1.weight, Requires Grad: False
Parameter: layer1.0.bn1.bias, Requires Grad: False
Parameter: layer1.0.conv2.weight, Requires Grad: False
Parameter: layer1.0.bn2.weight, Requires Grad: False
Parameter: layer1.0.bn2.bias, Requires Grad: False
Parameter: layer1.0.conv3.weight, Requires Grad: False
Parameter: layer1.0.bn3.weight, Requires Grad: False
Parameter: layer1.0.bn3.bias, Requires Grad: False
Parameter: layer1.0.downsample.0.weight, Requires Grad: False
Parameter: layer1.0.downsample.1.weight, Requires Grad: False
Parameter: layer1.0.downsample.1.bias, Requires Grad: False
Parameter: layer1.1.conv1.weight, Requires Grad: False
Parameter: layer1.1.bn1.weight, Requires Grad: False
Parameter: layer1.1.bn1.bias, Requires Grad: False
Parameter: layer1.1.conv2.weight, Requires Grad:

In [89]:
state_dict.keys()

dict_keys(['conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'layer1.0.conv1.weight', 'layer1.0.bn1.weight', 'layer1.0.bn1.bias', 'layer1.0.bn1.running_mean', 'layer1.0.bn1.running_var', 'layer1.0.bn1.num_batches_tracked', 'layer1.0.conv2.weight', 'layer1.0.bn2.weight', 'layer1.0.bn2.bias', 'layer1.0.bn2.running_mean', 'layer1.0.bn2.running_var', 'layer1.0.bn2.num_batches_tracked', 'layer1.0.conv3.weight', 'layer1.0.bn3.weight', 'layer1.0.bn3.bias', 'layer1.0.bn3.running_mean', 'layer1.0.bn3.running_var', 'layer1.0.bn3.num_batches_tracked', 'layer1.0.downsample.0.weight', 'layer1.0.downsample.1.weight', 'layer1.0.downsample.1.bias', 'layer1.0.downsample.1.running_mean', 'layer1.0.downsample.1.running_var', 'layer1.0.downsample.1.num_batches_tracked', 'layer1.1.conv1.weight', 'layer1.1.bn1.weight', 'layer1.1.bn1.bias', 'layer1.1.bn1.running_mean', 'layer1.1.bn1.running_var', 'layer1.1.bn1.num_batches_tracked', 'layer1.1.conv2.wei

In [101]:
data = torch.rand(128, 3, 7, 7)
model(data).shape

torch.Size([128, 2048])